# Processamento de Linguagem Natural - Trabalho Prático 1
### Thaís Ferreira da Silva - 2021092571

### Import

In [11]:
# Imports do gensim - para word2vec
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus

# Imports essenciais
import os
import random

### Preprocessamento do dados de treino

In [ ]:
text8_path = './text8'
questions_words_path = os.path.abspath('./mini-questions-words.txt')
corpus = Text8Corpus(text8_path)

# Visualizando as primeiras palavras do corpus
sentence = next(iter(corpus))
print(sentence[:50])



['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


### Treinamento do modelo Word2Vec

In [ ]:
def load_analogies(file_path):
    """Carrega as analogias do arquivo questions-words.txt."""
    analogies = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith(':'):  # Ignorar categorias
                continue
            words = line.split()
            if len(words) == 4:
                analogies.append(tuple(words))  # ('Athens', 'Greece', 'Baghdad', 'Iraq')
    return analogies

In [14]:
def generate_hyperparameter_combinations(param_grid):
    """Gera todas as combinações de hiperparâmetros"""
    from itertools import product
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    return combinations


In [ ]:
def train_and_save_model(corpus, params, output_dir):
    model_name = f"word2vec_vs{params['vector_size']}_win{params['window']}_sg{params['sg']}_ep{params['epochs']}"
    print(f"Treinando modelo: {model_name}")
    
    model = Word2Vec(
        sentences=corpus,
        vector_size=params['vector_size'],
        window=params['window'],
        sg=params['sg'],
        epochs=params['epochs'],
        workers=4
    )
    
    model_path = os.path.join(output_dir, f"{model_name}.model")
    model.save(model_path)
    print(f"Modelo salvo: {model_path}")
    
    return model

In [ ]:
def evaluate_analogies(model, analogies):
    """Realiza operações de analogia e verifica as palavras mais próximas"""
    results = []
    for analogy in analogies:
        try:
            # Verificando se todas as palavras estão no vocabulário do modelo
            missing_words = [word for word in analogy if word not in model.wv]
            if missing_words:
                results.append((analogy, None, None, False, f"Palavras ausentes: {', '.join(missing_words)}"))
                continue

            positive = [analogy[1], analogy[0]]  # France + Paris
            negative = [analogy[2]]  # Berlin
            result = model.wv.most_similar(positive=positive, negative=negative, topn=1)
            predicted_word = result[0][0]
            expected_word = analogy[3]  #Germany
            correct = predicted_word == expected_word
            results.append((analogy, predicted_word, expected_word, correct))
        
        except KeyError as e:
            results.append((analogy, None, None, False, f"Erro: {e}"))
    
    return results


In [ ]:
analogies = load_analogies(questions_words_path)
print(f"Total de analogias carregadas: {len(analogies)}")


output_dir = './word2vec_models'
os.makedirs(output_dir, exist_ok=True)

# Hiperparâmetros para o GridSearch
param_grid = {
    'vector_size': [50, 100, 200],      # Tamanho do vetor de palavras
    'window': [3, 5, 7],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [5, 10, 15],             # Número de iterações de treinamento
}

# Gerar combinações de hiperparâmetros
combinations = generate_hyperparameter_combinations(param_grid)
results_summary = []

Total de analogias carregadas: 22


In [ ]:
# for i, params in enumerate(combinations):
#     print(f"\nTreinando combinação {i+1}/{len(combinations)}: {params}\n")
#     model = train_and_save_model(corpus, params, output_dir)
    
#     # Avaliar o modelo usando as operações de analogia
#     analogy_results = evaluate_analogies(model, analogies)
    
#     # Exibir os resultados de cada analogia
#     for analogy, predicted, expected, correct in analogy_results[:5]:  # Mostra as 5 primeiras
#         print(f"Analogia: {analogy} | Previsto: {predicted} | Esperado: {expected} | Correto: {correct}")
    
#     # Salvar o desempenho geral dessa configuração
#     accuracy = sum(1 for _, _, _, correct in analogy_results if correct) / len(analogy_results)
#     results_summary.append((params, accuracy))


Vocabulário do modelo: ['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two']
Athena está no vocabulário? False
Greece está no vocabulário? False
Total de analogias: 22
Corretas: 0
Acurácia: 0.00%


### Avaliação do modelo treinado

In [ ]:
model = Word2Vec.load('./word2vec_models/word2vec_vs200_win7_sg1_ep15.model')

questions_words_path = './mini-questions-words.txt'
analogies = load_analogies(questions_words_path)

results = evaluate_analogies(model, analogies)

correct = sum(1 for _, _, _, c, *rest in results if c)
total = len(results)
accuracy = correct / total if total > 0 else 0

print(f"Total de analogias: {total}")
print(f"Corretas: {correct}")
print(f"Acurácia: {accuracy:.2%}")

print("\nPrimeiros 10 resultados:")
for analogy, predicted, expected, correct, *info in results[:10]:
    print(f"Analogia: {analogy} | Previsto: {predicted} | Esperado: {expected} | Correto: {correct}")
    if info:
        print(f"  Info extra: {info[0]}")


Análise de analogia (minúsculas): ['athens', 'greece', 'baghdad', 'iraq']
Análise de analogia (minúsculas): ['athens', 'greece', 'bangkok', 'thailand']
Análise de analogia (minúsculas): ['athens', 'greece', 'beijing', 'china']
Análise de analogia (minúsculas): ['athens', 'greece', 'berlin', 'germany']
Análise de analogia (minúsculas): ['athens', 'greece', 'bern', 'switzerland']
Análise de analogia (minúsculas): ['athens', 'greece', 'cairo', 'egypt']
Análise de analogia (minúsculas): ['athens', 'greece', 'canberra', 'australia']
Análise de analogia (minúsculas): ['athens', 'greece', 'hanoi', 'vietnam']
Análise de analogia (minúsculas): ['athens', 'greece', 'havana', 'cuba']
Análise de analogia (minúsculas): ['athens', 'greece', 'helsinki', 'finland']
Análise de analogia (minúsculas): ['athens', 'greece', 'islamabad', 'pakistan']
Análise de analogia (minúsculas): ['athens', 'greece', 'kabul', 'afghanistan']
Análise de analogia (minúsculas): ['athens', 'greece', 'london', 'england']
Análi